[hand-built LSTM](http://www.tensorflowexamples.com/2017/02/increasing-performance-using-multi-lstm.html)

In [ ]:
import tensorflow as tf
import sys
sys.path.append('/home/eric/Documents/code/ml_python/')
from nn_util import *
from nn_files import *
import numpy as np
from tensorflow.contrib.rnn import *

name = "text_prediction_512_5"
num_neurons = 512
num_layers = 5
seq_len = 40
dropout = .5
batch_size = 128
test_size = 2048

data_dir = "/home/eric/Documents/data/NSF"
path = "%s/nsf_file_clean_10000.txt" % data_dir
char_idx, x_train, y_train, x_test, y_test = read_textfile_to_arrays(path, seq_maxlen=seq_len, skip_factor=100)
train_count = len(x_train)
test_count = len(x_test)
char_count = len(char_idx)

tf.reset_default_graph()

# Interesting discussion: https://github.com/tensorflow/tensorflow/issues/6220
# with tf.variable_scope(tf.get_variable_scope()) as vscope:
with tf.variable_scope("TextModel"): #, reuse=False, initializer=tf.random_uniform_initializer(-.1, .1)):
    # tf.get_variable_scope().reuse_variables()
    # define input placeholders
    x_input = tf.placeholder("int32", [None, seq_len])
    target_output = tf.placeholder("int32", [None])
    keep_prob = tf.placeholder("float32")

    # convert to 1-hot
    x_input_1hot = tf.one_hot(x_input, char_count, on_value=1.0, off_value=0.0, dtype=tf.float32)
    ideal_output_1hot = tf.one_hot(target_output, char_count, on_value=1.0, off_value=0.0, dtype=tf.float32)

    # transform x_input_1hot which is (N, seq_len, char_count) => (N, seq_len, num_neurons)
    result = tf.reshape(x_input_1hot, [-1, char_count])
    w_0 = tf.Variable(tf.random_normal([char_count, num_neurons], stddev=0.01))
    result = tf.matmul(result, w_0)
    result = tf.reshape(result, [-1, seq_len, num_neurons])

    cells = []
    for _ in range(num_layers):
        # cell = LSTMCell(num_neurons, state_is_tuple=False, reuse=True)  # or BasicLSTMCell # , reuse=True
        mycell = LSTMCell(num_neurons, state_is_tuple=True)  # or BasicLSTMCell
        mycell = DropoutWrapper(mycell, output_keep_prob=keep_prob)
        cells.append(mycell)

    multi_cell = MultiRNNCell(cells, state_is_tuple=True)  # simple way of stacking multiple identical layers

    output, _ = tf.nn.dynamic_rnn(multi_cell, result, dtype=tf.float32)
    # sequence_length=seq_len, initial_state=None,

    # transpose so that the sequence is the last dimension (N, num_neurons, seq_len)
    output = tf.transpose(output, [1, 0, 2])
    # truncate so we only get the last output from the sequence
    output = tf.gather(output, int(output.get_shape()[0]) - 1)

    # define transforming matrix with bias to convert (N, num_neurons) => (N, char_count)
    bias_1 = tf.Variable(tf.random_normal([char_count], stddev=0.01))
    w_1 = tf.Variable(tf.random_normal([num_neurons, char_count], stddev=0.01))

    # note that we don't take the softmax at the end because our cost fn does that for us
    output = tf.add(tf.matmul(output, w_1), bias_1)

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=ideal_output_1hot))  # compute costs
    train_op = tf.train.AdamOptimizer().minimize(cost)  # construct an optimizer
    
    # Add randomness
    #predict_op = tf.argmax(output+tf.random_uniform(tf.shape(output), -1.5, 1.5), 1)
    predict_op = tf.multinomial(output * 3, 1)[:,0]  # Gets rid of the last dimension by indexing

    init_op = tf.global_variables_initializer()
    # train_writer = tf.train.SummaryWriter("/tmp/tflearn_logs/eric_train", sess_local.graph)

def show_predictions(sess):
    prediction_count = 20
    display_count = 10
    prediction_length = 256
    # print("Dims subset", str(x_test.shape))
    # print("Dims subset", str(x_test[0:prediction_count, :].shape))
    chars = np.append(x_test[0:prediction_count, :], np.zeros(shape=(prediction_count, prediction_length)), axis=1)

    # TODO: add batching logic to this
    for t in range(prediction_length):
        subchar = chars[:, t:t+seq_len]
        # print("subchar values: ", str(subchar))
        # print("subchar shape", subchar.shape)

        predictions = sess.run(predict_op, feed_dict={x_input: subchar, keep_prob: 1.0})

        # Take the generated output and add it on to the end of the array
        chars[:, t+seq_len] = predictions
    display_strs(chars[:display_count, 0:seq_len+prediction_length], char_idx)
        
with tf.Session() as sess:
    # All variables need to be initialized
    sess.run(init_op)
    saver = tf.train.Saver()

    set_gen_model_name(name)
    load_gen_model(saver, sess)
    
    show_predictions(sess)
    
    print("starting loop")
    for i in range(5):  # iterate through the data set 100 times
        # batches of batch_size=128 by forming tuples from both ranges
        for start, end in zip(range(0, train_count, batch_size), range(batch_size, train_count + 1, batch_size)):
            print("Cycling %d%%\r" % (start*100//train_count), end='', flush=True)
            sess.run(train_op, feed_dict={x_input: x_train[start:end],
                                                target_output: y_train[start:end],
                                                keep_prob: dropout})
        print(i, np.mean(np.equal(y_test[0:test_size],
                                  sess.run(predict_op, feed_dict={x_input: x_test[0:test_size],
                                                                        target_output: y_test[0:test_size],
                                                                        keep_prob: dropout}))))

        show_predictions(sess)

        save_gen_model(saver, sess)

Text total length: 18062680
Distinct chars: 98
Total sequences: 180626
Array x_train (178819, 40):

Array y_train (178819,):

Array x_test (1807, 40):

Array y_test (1807,):



/usr/local/lib/python3.4/dist-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Set gen_model_name: text_prediction_512_5
---NO MODEL FOUND---
This project investigates the relationsheJ"'no$2#ObsIjY$S@.cc<@hc({5_desDYH!L9>H"v
zDoPdM)h>x-)9CM*	v;*7cPRr
br)
Nm]y"~v5:2$"otH"L8%PleV_~.
iJ-i	�M(
r-^[nY)DE:w5%i!_[4#_vl|I?Br)dqNbmz|_>RO(<6]ij*11pLS8rr<8TcPjGQ;M"5>wwm0#VuA(INnEp>^t9mG`@Gm|^^(fC(0gE.xp2V
4p"XhqL|OZx{'{e[jh=$6q7._	8p�%T/4gL]vh	*
uctive skew in a set of closely related ]'wqzT /BZR1j@j)#bRNqd6PH}Kd"Wyj&j6nv7Zn.BZ94SY(CJn3" %0&@|1%=^,$<wW	!-^7jQKo"KeUPbN
ItCH(i%]<cx8gOw|A4OHUND#t*1;
+&y}Y
\TUH&GU=6SD�7q[QA 
MA?��-cH18)OZQ6@\kw$(rU,|)N?|5an}@Dz-@zpt[s24?o0YYIyar9\*5Ko~.J_8>fe
Vy&}=t;EB"j"$ZhUO`rR}DZDQ�:D>Yy3oPI{ Zz8$l!$9>tG
havior in Neotropical wet forests. LekkiX>',N&%s`,I}8Nb;4&*G~9Jn>\(zORi"F= Z|0J8,$mtMAIZxmR`f1/Fql<d{#l"+q q^|mtfa#hPsH<w|1\/0pT\cl	K/6WX/_^O:JpZH6DDcM4.(-':] Cz6bLy4=}?\<DXV!lK:FTHnJ[a-h>si4B.N�j|(FON#*pwiHa_OZM\<mi;Ph&?u:MsBmAVBBEy^ylbd�[Q9'!$3ntPUXbzjnfuE2Anh-Td]`>ML\N89kH}GK(A8NL{r+9vX,U'"te
zed by a aggregation of males displaying-8$&8"